In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv1D, Flatten, MaxPooling1D, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import metrics as metrics
from collections import OrderedDict, defaultdict

# import from other files
from position import Position


ticker_list = [
    'AAPL',
    'MSFT',
    'AMZN',
    'GOOGL',
    'NVDA',
    'TSLA',
    'META',
    'GOOG',
    'XOM',
    'UNH',
    'JPM',
    'LLY',
    'JNJ',
    'V',
    'PG',
    'AVGO',
    'MA',
    'HD',
    'CVX',
    'ABBV',
    'MRK',
    'COST',
    'PEP',
    'WMT',
    'ADBE',
    'CSCO',
    'KO',
    'CRM',
    'TMO',
    'ACN'
]


class PortfolioTrader():

    def __init__(self, quantity=50, cash=1000000, max_active_positions=30 * 10, percent_slippage=0.0005, trade_fee=1, stop_loss=0.15):

        print("Initializing PortfolioTrader...")
        self.models_dict = {ticker: pd.read_csv("ML Models/data/" + ticker + ".csv",
                                                index_col="Date", parse_dates=True) for ticker in ticker_list}
        self.logged_positions = {ticker: []
                                 for ticker in ticker_list}  # : Ticker: List[Position]

        self.quantity = quantity
        self.stop_loss = stop_loss

        # Cash series
        self.cash = cash
        self.cash_history = {}  # date: cash

        # Stock allocation of portfolio
        # ticker: total value of shares
        self.stock_allocation = {ticker: 0 for ticker in ticker_list}
        self.current_positions = 0

        # Trade parameters
        self.max_active_positions = max_active_positions
        self.percent_slippage = percent_slippage
        self.trade_fee = trade_fee

        self.simulation_finished = False

        # Dates
        self.current_date = '2011-01-12'
        self.last_date = '2011-01-12'
        self.start_date = '2011-01-12'
        self.end_date = "2020-12-31"

        # Result Summaries data frame
        self.metrics = pd.DataFrame(columns=['Stop Loss', 'Quantity Shares Traded', 'Max Active Positions',
                                             'Sharpe Ratio', 'Max Drawdown', 'Annualized return'])

    def add_to_history(self, position: Position):
        _log = self.logged_positions
        assert not position in _log, 'Recorded the same position twice.'
        assert position.is_closed, 'Position is not closed.'
        self.logged_positions.add(position)
        self.position_history.append(position)
        self.last_date = max(self.last_date, position.last_date)

    def record_cash(self, date, cash):
        self.cash_history[datetime.datetime.strptime(date, '%Y-%m-%d')] = cash
        self.last_date = max(self.last_date, date)

    def check_to_open_long_positions(self, ticker, price):

        # if ticker in self.stock_allocation:
        #    if self.stock_allocation[ticker] >= self.max_allocation:
        #        return False

        # Check if we have enough cash to buy
        # if self.stock_allocation[ticker] + 10 * price >= self.max_allocation or self.cash < 10 * price or self.current_positions >= self.max_active_positions:
        if self.cash < self.quantity * price or self.current_positions >= self.max_active_positions:
            # print("Not enough cash and can't open long position")
            return False
        else:
            return True

    def check_to_open_short_positions(self, ticker, price):

        # if ticker in self.stock_allocation:
        #    if self.stock_allocation[ticker] < -self.max_allocation:
        #        return False

        # Check if we have enough cash to buy
        # if self.stock_allocation[ticker] - 10 * price < -self.max_allocation or self.cash < 10 * price or self.current_positions >= self.max_active_positions:
        if self.cash < self.quantity * price or self.current_positions >= self.max_active_positions:
            # print("Not enough cash and can't open short position")
            return False

        else:
            # Open position
            return True

    def simulate_day(self, date):

        current_cash = self.cash
        # print(f"Current Positions: {self.current_positions}")
        for ticker in ticker_list:

            # Access row by date index
            try:
                current_data = self.models_dict[ticker].loc[self.current_date]
            except KeyError:
                print(f"No data for {ticker} on {date}")
                continue

            signal = current_data["position"]
            current_price = current_data["Close"]
            # print(f"Simulating {ticker} on {date}, signal = {signal}")

            # Update current price in position histories and logged positions
            new_posistions = []
            for position in self.logged_positions[ticker]:
                # print(f"Updating {ticker} position")
                if position.is_active:
                    position.record_price_update(
                        self.current_date, current_price)

                if position.is_active and position.stop_loss_hit() and position.type == "long" or (position.is_active and position.type == "long") and signal == -1:
                    position.exit(self.current_date, current_price)
                    self.current_positions -= 1

                    # Update stock allocation
                    current_stock_allocation = self.stock_allocation[ticker]
                    # print(f"Current stock allocation: {current_stock_allocation}")
                    current_stock_allocation -= position.last_price * position.shares
                    self.stock_allocation[ticker] = current_stock_allocation
                    # print(f"New stock allocation: {current_stock_allocation}")

                    # Update cash
                    # print(f"Current cash: {current_cash}")
                    current_cash += position.last_price * position.shares - self.trade_fee
                    # print(f"New cash: {current_cash}")

                elif position.is_active and position.stop_loss_hit() and position.type == "short" or (position.is_active and position.type == "short" and signal == 1):
                    position.exit(self.current_date, current_price)
                    self.current_positions -= 1

                    # Update stock allocation
                    current_stock_allocation = self.stock_allocation[ticker]
                    # print(
                    #    f"Current stock allocation: {current_stock_allocation}")
                    current_stock_allocation -= position.last_price * position.shares
                    self.stock_allocation[ticker] = current_stock_allocation
                    # print(f"New stock allocation: {current_stock_allocation}")

                    # Update cash
                    current_cash -= position.last_price * position.shares - self.trade_fee

                if position.is_active:
                    new_posistions.append(position)

            # Update positions after closing
            self.logged_positions[ticker] = new_posistions

            if signal == 1:
                # print("Long signal for " + ticker)
                canOpen = self.check_to_open_long_positions(
                    ticker, current_price)

                if canOpen:

                    current_stock_allocation = self.stock_allocation[ticker]

                    # Open new Position and update position logs
                    position = Position(
                        ticker, self.current_date, "long", current_price, self.quantity, self.stop_loss)

                    assert not position.is_closed, 'Position is not closed.'
                    self.logged_positions[ticker].append(position)
                    self.current_positions += 1

                    # Update Cash and stock allocation
                    current_cash -= self.quantity * current_price * \
                        (1 + self.percent_slippage) + self.trade_fee
                    current_stock_allocation += self.quantity * \
                        current_price * (1 + self.percent_slippage)

                    self.stock_allocation[ticker] = current_stock_allocation

                    # print(
                    #    f"Opened long position in {ticker} at {current_price} on {date}, Quantity: {self.quantity}")

            elif signal == -1:
                canShort = self.check_to_open_short_positions(
                    ticker, current_price)

                if canShort:

                    current_stock_allocation = self.stock_allocation[ticker]

                    # Open new Position and update position logs
                    position = Position(
                        ticker, self.current_date, "short", current_price, self.quantity, self.stop_loss)

                    self.logged_positions[ticker].append(position)
                    self.current_positions += 1

                    # Update Cash and stock allocation
                    current_cash += self.quantity * current_price * \
                        (1 - self.percent_slippage) - self.trade_fee
                    current_stock_allocation += self.quantity * \
                        current_price * (1 - self.percent_slippage)
                    self.stock_allocation[ticker] = current_stock_allocation

                    # print(
                    #    f"Opened short position in {ticker} at {current_price} on {date}, Quantity: {self.quantity}")

        self.cash = current_cash
        # print(f"EOD Positions: {self.current_positions}")

    def simulate(self):

        while not self.simulation_finished and datetime.datetime.strptime(self.current_date, '%Y-%m-%d') < datetime.datetime.strptime(self.end_date, '%Y-%m-%d'):

            self.simulate_day(self.current_date)
            # print(f"Current date: {self.current_date} + Cash: {self.cash}")

            self.record_cash(self.current_date, self.cash)

            self.current_date = (datetime.datetime.strptime(
                self.current_date, '%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

        # self.close_profit_positions()
        print("Simulation finished")

    def close_all_positions(self):

        for ticker in ticker_list:
            for position in self.logged_positions[ticker]:
                if position.is_active and position.type == "long":
                    position.exit(self.current_date, position.last_price)

                    # Update stock allocation
                    current_stock_allocation = self.stock_allocation[ticker]
                    # print(f"Current stock allocation: {current_stock_allocation}")
                    current_stock_allocation -= position.last_price * position.shares
                    self.stock_allocation[ticker] = current_stock_allocation
                    # print(f"New stock allocation: {current_stock_allocation}")

                    # Update cash
                    # print(f"Current cash: {self.cash}")
                    self.cash += position.last_price * position.shares - self.trade_fee
                    # print(f"New cash: {self.cash}")

                elif position.is_active and position.type == "short":
                    position.exit(self.current_date, position.last_price)

                    # Update stock allocation
                    current_stock_allocation = self.stock_allocation[ticker]
                    # print(f"Current stock allocation: {current_stock_allocation}")
                    current_stock_allocation -= position.last_price * position.shares
                    self.stock_allocation[ticker] = current_stock_allocation
                    # print(f"New stock allocation: {current_stock_allocation}")

                    # Update cash
                    # print(f"Current cash: {self.cash}")
                    self.cash -= position.last_price * position.shares - self.trade_fee
                    # print(f"New cash: {self.cash}")

    def close_profit_positions(self):

        for ticker in ticker_list:
            for position in self.logged_positions[ticker]:
                if position.is_active and position.type == "long" and position.last_price > position.entry_price:
                    position.exit(self.current_date, position.last_price)

                    # Update stock allocation
                    current_stock_allocation = self.stock_allocation[ticker]
                    # print(
                    #    f"Current stock allocation: {current_stock_allocation}")
                    current_stock_allocation -= position.last_price * position.shares
                    self.stock_allocation[ticker] = current_stock_allocation
                    # print(f"New stock allocation: {current_stock_allocation}")

                    # Update cash
                    # print(f"Current cash: {self.cash}")
                    self.cash += position.last_price * position.shares - self.trade_fee
                    # print(f"New cash: {self.cash}")

                elif position.is_active and position.type == "short" and position.last_price < position.entry_price:
                    position.exit(self.current_date, position.last_price)

                    # Update stock allocation
                    current_stock_allocation = self.stock_allocation[ticker]
                    # print(
                    #    f"Current stock allocation: {current_stock_allocation}")
                    current_stock_allocation -= position.last_price * position.shares
                    self.stock_allocation[ticker] = current_stock_allocation
                    # print(f"New stock allocation: {current_stock_allocation}")

                    # Update cash
                    # print(f"Current cash: {self.cash}")
                    self.cash -= position.last_price * position.shares - self.trade_fee
                    # print(f"New cash: {self.cash}")

    def plot_cash(self):
        cash_history = pd.Series(self.cash_history)
        cash_history.plot(figsize=(20, 10))
        plt.title('Cash over time', fontsize=20)
        plt.xlabel('Date', fontsize=20)
        plt.ylabel('Cash', fontsize=20)
        plt.grid(axis='both')
        plt.show()

    def calculate_metrics(self):
        cash_history = pd.Series(self.cash_history)

        # Calculate annualized return
        annualized_return = ((self.cash - 1000000) / 1000000) / (datetime.datetime.strptime(
            self.end_date, '%Y-%m-%d').year - datetime.datetime.strptime(self.start_date, '%Y-%m-%d').year)

        # Calculate Sharpe Ratio
        sharpe = metrics.calculate_sharpe_ratio(cash_history)

        # Calculate Max Drawdown
        max_drawdown = metrics.calculate_max_drawdown(cash_history)

        # Calculate Sortino Ratio
        sortino = metrics.calculate_sortino_ratio(cash_history)

        # Add to metrics data frame

        # ['Stop Loss', 'Quantity Shares Traded', 'Max Active Positions',
        #                                   'Sharpe Ratio', 'Max Drawdown', 'Annualized return']
        self.metrics = pd.concat([self.metrics, pd.DataFrame({'Stop Loss': [self.stop_loss],
                                                              'Quantity Shares Traded': [self.quantity],
                                                              'Max Active Positions': [self.max_active_positions / 30],
                                                              'Sharpe Ratio': [sharpe],
                                                              'Max Drawdown': [max_drawdown],
                                                              'Annualized return': [annualized_return]})],
                                 axis=0, ignore_index=True)

    def print_summary(self):

        print(f"Total profit: {self.cash - 1000000}")
        print(f"Total cash: {self.cash}")

        # Calculate annualized return
        print(f"Annualized return: {self.metrics['Annualized return']}")

        # Calculate Sharpe Ratio
        print(f"Sharpe Ratio: {self.metrics['Sharpe Ratio']}")

        # Calculate Max Drawdown
        print(f"Max Drawdown: {self.metrics['Max Drawdown']}")

        # Calculate Sortino Ratio
        print(f"Sortino Ratio: {self.metrics['Sortino Ratio']}")

    def plot_profit(self):
        books = self.books
        books.index = books['Date']

        books['Profit'].plot(figsize=(20, 10))
        plt.title('Profit over time', fontsize=20)
        plt.xlabel('Date', fontsize=20)
        plt.ylabel('Profit', fontsize=20)
        plt.grid(axis='both')
        plt.show()


In [2]:
metrics = pd.DataFrame(columns=['Stop Loss', 'Quantity Shares Traded', 'Max Active Positions',
                      'Sharpe Ratio', 'Max Drawdown', 'Annualized return'])


In [3]:
metrics

,Stop Loss,Quantity Shares Traded,Max Active Positions,Sharpe Ratio,Max Drawdown,Annualized return


In [13]:
metrics = pd.concat([metrics, pd.DataFrame({'Stop Loss': [0.0],
                                  'Quantity Shares Traded': [0.0],
                                  'Max Active Positions': [0.0],
                                  'Sharpe Ratio': [1.0],
                                  'Max Drawdown': [0.0],
                                  'Annualized return': [0.0]})],
          axis=0, ignore_index=True)


In [14]:
metrics

,Stop Loss,Quantity Shares Traded,Max Active Positions,Sharpe Ratio,Max Drawdown,Annualized return
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
overview = pd.DataFrame(columns=['Stop Loss', 'Quantity Shares Traded', 'Max Active Positions',
                                 'Sharpe Ratio', 'Max Drawdown', 'Annualized return'])


In [18]:
overview = pd.concat([overview, metrics], axis=0, ignore_index=True)


In [19]:
overview

,Stop Loss,Quantity Shares Traded,Max Active Positions,Sharpe Ratio,Max Drawdown,Annualized return
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
overview.sort_values(by='Sharpe Ratio', ascending=False, inplace=True)


In [22]:
overview

,Stop Loss,Quantity Shares Traded,Max Active Positions,Sharpe Ratio,Max Drawdown,Annualized return
2,0.0,0.0,0.0,1.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
